In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
sample = " if you want you"
idx2char = list(set(sample))
char2idx = {c: i for i, c in enumerate(idx2char)}

dic_size = len(char2idx)
hidden_size = len(char2idx)
num_classes = len(char2idx)#output_size
batch_size = 1
sequence_length = len(sample) - 1
learning_rate = 0.1

sample_idx = [char2idx[c] for c in sample]
x_data = [sample_idx[:-1]]
y_data = [sample_idx[1:]]

X = tf.placeholder(tf.int32, [None, sequence_length])
Y = tf.placeholder(tf.int32, [None, sequence_length])

x_one_hot = tf.one_hot(X, num_classes)
cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True)
initial_state = cell.zero_state(batch_size, tf.float32)
outputs, _states = tf.nn.dynamic_rnn(cell, x_one_hot, initial_state=initial_state, dtype=tf.float32)

#FC layer
X_for_fc = tf.reshape(outputs, [-1, hidden_size])
outputs = tf.contrib.layers.fully_connected(X_for_fc, num_classes, activation_fn=None)

outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])

#weights : 전부 1로 만듬
weights = tf.ones([batch_size, sequence_length])
sequence_loss = tf.contrib.seq2seq.sequence_loss(logits=outputs, targets=Y, weights=weights)
loss = tf.reduce_mean(sequence_loss)
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

prediction = tf.argmax(outputs, axis=2)

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(50):
        l, _ = sess.run([loss, train], feed_dict={X: x_data, Y: y_data})
        result = sess.run(prediction, feed_dict={X: x_data})

        result_str = [idx2char[c] for c in np.squeeze(result)]

        print(i, "loss: ", l, "Prediction:", ''.join(result_str))

## long sentence

In [1]:
from __future__ import print_function

import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn
tf.set_random_seed(777)

In [2]:
sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")

char_set = list(set(sentence))
char_dic = {w: i for i, w in enumerate(char_set)}

data_dim = len(char_set)
hidden_size = len(char_set)
num_classes = len(char_set)
sequence_length = 10
learning_rate = 0.1

dataX = []
dataY = []
for i in range(0, len(sentence) - sequence_length):
    x_str = sentence[i : i + sequence_length]
    y_str = sentence[i + 1 : i + sequence_length + 1]
    print(i, x_str, '->', y_str)
    
    x = [char_dic[c] for c in x_str]
    y = [char_dic[c] for c in y_str]
    
    dataX.append(x)
    dataY.append(y)
    
batch_size = len(dataX)

X = tf.placeholder(tf.int32, [None, sequence_length])
Y = tf.placeholder(tf.int32, [None, sequence_length])

#one_hot_encoding
X_one_hot = tf.one_hot(X, num_classes)
print(X_one_hot)

def lstm_cell():
    cell = rnn.BasicLSTMCell(hidden_size, state_is_tuple=True)
    return cell

#cell 여러개 쌓기
multi_cells = rnn.MultiRNNCell([lstm_cell() for _ in range(2)], state_is_tuple=True)

outputs, _states = tf.nn.dynamic_rnn(multi_cells, X_one_hot, dtype=tf.float32)

#FC layer
X_for_fc = tf.reshape(outputs, [-1, hidden_size])
outputs = tf.contrib.layers.fully_connected(X_for_fc, num_classes, activation_fn=None)

outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])

weights = tf.ones([batch_size, sequence_length])

sequence_loss = tf.contrib.seq2seq.sequence_loss(logits=outputs, targets=Y, weights=weights)
mean_loss = tf.reduce_mean(sequence_loss)
train_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(mean_loss)



0 if you wan -> f you want
1 f you want ->  you want 
2  you want  -> you want t
3 you want t -> ou want to
4 ou want to -> u want to 
5 u want to  ->  want to b
6  want to b -> want to bu
7 want to bu -> ant to bui
8 ant to bui -> nt to buil
9 nt to buil -> t to build
10 t to build ->  to build 
11  to build  -> to build a
12 to build a -> o build a 
13 o build a  ->  build a s
14  build a s -> build a sh
15 build a sh -> uild a shi
16 uild a shi -> ild a ship
17 ild a ship -> ld a ship,
18 ld a ship, -> d a ship, 
19 d a ship,  ->  a ship, d
20  a ship, d -> a ship, do
21 a ship, do ->  ship, don
22  ship, don -> ship, don'
23 ship, don' -> hip, don't
24 hip, don't -> ip, don't 
25 ip, don't  -> p, don't d
26 p, don't d -> , don't dr
27 , don't dr ->  don't dru
28  don't dru -> don't drum
29 don't drum -> on't drum 
30 on't drum  -> n't drum u
31 n't drum u -> 't drum up
32 't drum up -> t drum up 
33 t drum up  ->  drum up p
34  drum up p -> drum up pe
35 drum up pe -> rum up peo
36

In [3]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for i in range(500):
    _, l, results = sess.run(
        [train_op, mean_loss, outputs], feed_dict={X: dataX, Y: dataY})
    for j, result in enumerate(results):
        index = np.argmax(result, axis=1)
        print(i, j, ''.join([char_set[t] for t in index]), l)


0 0 mmmkkmwww' 3.21555
0 1 .kkkkwww'' 3.21555
0 2 wwmwwwwwww 3.21555
0 3 kkkwwu''ll 3.21555
0 4 m ww'''lll 3.21555
0 5 yyw''''lll 3.21555
0 6 wnrrrkllll 3.21555
0 7 idddklllll 3.21555
0 8 eeefllllll 3.21555
0 9 slllllllll 3.21555
0 10 lllllllmmm 3.21555
0 11 wwmmmwmmmm 3.21555
0 12 lllllmmmww 3.21555
0 13 mmmwwwmwww 3.21555
0 14 wwyywwwwww 3.21555
0 15 tyyttwwwww 3.21555
0 16 yywmwrwwww 3.21555
0 17 mmkkrrwwr' 3.21555
0 18 tkkrrwwrr' 3.21555
0 19 kkrrrrrrrw 3.21555
0 20 wnnwwuuuww 3.21555
0 21 e'''uuuuww 3.21555
0 22 ww uuuwwww 3.21555
0 23 lsssssukww 3.21555
0 24 ukuuuukmwm 3.21555
0 25 milgnmmmmm 3.21555
0 26 iggnmmmmmm 3.21555
0 27 l mmmmmmmm 3.21555
0 28 wnkmmmmmmm 3.21555
0 29 kkkkkmkkk  3.21555
0 30 mloll k    3.21555
0 31 selwkkk  w 3.21555
0 32 .kkkkkkkww 3.21555
0 33 llkkkkwwww 3.21555
0 34 wnkkkwwwww 3.21555
0 35 kkkkkwwwww 3.21555
0 36 k   wwwwww 3.21555
0 37 ywwwwwwwww 3.21555
0 38 kwwwwwwwww 3.21555
0 39 wwyyyywwww 3.21555
0 40 yyyyyyywww 3.21555
0 41 innnnywwww 3.21555
0 

3 148 eooooooooo 3.02148
3 149 oooooooooo 3.02148
3 150 oooooooooo 3.02148
3 151 ooonoooooo 3.02148
3 152 onnnnnnooo 3.02148
3 153 aannnnoooo 3.02148
3 154 oooooooooo 3.02148
3 155 oonnoooooo 3.02148
3 156 annnnnoooo 3.02148
3 157 aooooooooo 3.02148
3 158 oooooooooo 3.02148
3 159 oonooooooo 3.02148
3 160 annooooooo 3.02148
3 161 aooooooooo 3.02148
3 162 oooooooooo 3.02148
3 163 oooooooooo 3.02148
3 164 anoooooooo 3.02148
3 165 oooooooooo 3.02148
3 166 oooooooooo 3.02148
3 167 oooooooooo 3.02148
3 168 oooooooooo 3.02148
3 169 oooooooooo 3.02148
4 0 ooooonnnnn 2.91584
4 1 oooonnnnnn 2.91584
4 2 ooonnnnnnn 2.91584
4 3 oonnnnnnnn 2.91584
4 4 onnnnnnnnn 2.91584
4 5 nnnnnnnnnn 2.91584
4 6 oonnnnnnnn 2.91584
4 7 onnnnnnonn 2.91584
4 8 nnnnooonnn 2.91584
4 9 oooooooonn 2.91584
4 10 ooooooonnn 2.91584
4 11 ooooonnnnn 2.91584
4 12 oooonnnnnn 2.91584
4 13 ooonnnnnnn 2.91584
4 14 oonnnnnnnn 2.91584
4 15 ennnnnnnna 2.91584
4 16 nnnnnnnnnn 2.91584
4 17 oonnnnnnnn 2.91584
4 18 onnnnnnnnn 2.91584
4 19

6 90   eee  ee  2.75891
6 91  eee  ee   2.75891
6 92 ee   ee    2.75891
6 93 e   e   ss 2.75891
6 94    e   sss 2.75891
6 95   ees  sss 2.75891
6 96  eee   sss 2.75891
6 97 e    sssss 2.75891
6 98    sssssss 2.75891
6 99    sssssss 2.75891
6 100    sssssss 2.75891
6 101     ssssss 2.75891
6 102      sss s 2.75891
6 103     sss  s 2.75891
6 104     d      2.75891
6 105          s 2.75891
6 106            2.75891
6 107 e          2.75891
6 108            2.75891
6 109            2.75891
6 110          s 2.75891
6 111            2.75891
6 112            2.75891
6 113 e          2.75891
6 114       e    2.75891
6 115 eeeeee  ee 2.75891
6 116    ee   ee 2.75891
6 117   ee   ee  2.75891
6 118   e   ee   2.75891
6 119 ee  eeeeee 2.75891
6 120 e   ee eee 2.75891
6 121    ee eeee 2.75891
6 122   eeeeeeee 2.75891
6 123  ee eeeeee 2.75891
6 124 e  eeeeeee 2.75891
6 125    e eee e 2.75891
6 126     ee  ee 2.75891
6 127 eeeeee eee 2.75891
6 128 e eee  ee  2.75891
6 129  eee  ee e 2.75891
6 130 ee  

9 128   he   ee  2.53897
9 129  oe   ee   2.53897
9 130  e   ee he 2.53897
9 131     ee ee  2.53897
9 132    ee  e   2.53897
9 133   ee  e    2.53897
9 134  oe  e   h 2.53897
9 135    he  hhe 2.53897
9 136   he   ho  2.53897
9 137  ee   ho   2.53897
9 138     hhh  h 2.53897
9 139    hhh  he 2.53897
9 140    hh  he  2.53897
9 141   hh  he   2.53897
9 142   h  he    2.53897
9 143     he   e 2.53897
9 144    he   e  2.53897
9 145   he   e   2.53897
9 146  oe   e    2.53897
9 147  e  ee     2.53897
9 148     e      2.53897
9 149    e       2.53897
9 150            2.53897
9 151            2.53897
9 152    d       2.53897
9 153      i     2.53897
9 154            2.53897
9 155            2.53897
9 156            2.53897
9 157            2.53897
9 158            2.53897
9 159         ho 2.53897
9 160         oe 2.53897
9 161        ho  2.53897
9 162       hh   2.53897
9 163      ho  h 2.53897
9 164     he  he 2.53897
9 165   hhe  he  2.53897
9 166   oeo he   2.53897
9 167   ho he  h 2.53897


12 128   oe   e   2.17075
12 129  oe   ee o 2.17075
12 130  e   e  oe 2.17075
12 131     oe oe  2.17075
12 132    oe oe   2.17075
12 133   oe od  t 2.17075
12 134  oe oe  to 2.17075
12 135   toe  too 2.17075
12 136   od  too  2.17075
12 137  oe  too t 2.17075
12 138 e  ttoo to 2.17075
12 139   ttoo toe 2.17075
12 140  ttoo toe  2.17075
12 141  toe toe   2.17075
12 142  oe toe    2.17075
12 143    toe   e 2.17075
12 144   toe   e  2.17075
12 145   oe   e   2.17075
12 146  oe   d  s 2.17075
12 147  e   e  s  2.17075
12 148     d  s n 2.17075
12 149   od  n ni 2.17075
12 150   d  d ni  2.17075
12 151    nd nn   2.17075
12 152   nd nn    2.17075
12 153  ni ni   n 2.17075
12 154 e  ni   ns 2.17075
12 155   si   ns  2.17075
12 156 nni   ns   2.17075
12 157 nn   ns    2.17075
12 158     ns   t 2.17075
12 159    ns   to 2.17075
12 160   ns   too 2.17075
12 161       too  2.17075
12 162    d too t 2.17075
12 163     too to 2.17075
12 164 n  too toe 2.17075
12 165   to  toe  2.17075
12 166  to  

...

487 150 tndless im 0.228768
487 151  dless imm 0.228768
487 152 d ess imme 0.228768
487 153  ess immen 0.228768
487 154 ecs immens 0.228768
487 155  s immensi 0.228768
487 156 s immensit 0.228768
487 157 sammensity 0.228768
487 158 tmmensity  0.228768
487 159 tmensity o 0.228768
487 160  ensity of 0.228768
487 161  nsity of  0.228768
487 162  dity of t 0.228768
487 163 dity of th 0.228768
487 164 sgy of the 0.228768
487 165 ty of the  0.228768
487 166 h of the s 0.228768
487 167  of the se 0.228768
487 168 tf the sea 0.228768
487 169   the sea. 0.228768
488 0 m you want 0.228767
488 1 oyou want  0.228767
488 2 tou want t 0.228767
488 3 ou want to 0.228767
488 4 n want to  0.228767
488 5 mwant to b 0.228767
488 6 tont to bu 0.228767
488 7 ont to bui 0.228767
488 8 nd to buil 0.228767
488 9 d to build 0.228767
488 10 hto build  0.228767
488 11 to build a 0.228767
488 12 h luild a  0.228767
488 13 nluild a s 0.228767
488 14 tutld a sh 0.228767
488 15 utld a shi 0.228767
488 16 mld a ship 

491 32 t arum up  0.228763
491 33 harum up p 0.228763
491 34 toum up pe 0.228763
491 35  um up peo 0.228763
491 36  m up peop 0.228763
491 37 i up peopl 0.228763
491 38  tp people 0.228763
491 39 tp people  0.228763
491 40 i people t 0.228763
491 41 epeople to 0.228763
491 42 teople tog 0.228763
491 43 eople toge 0.228763
491 44 nple toget 0.228763
491 45 nle togeth 0.228763
491 46 ee togethe 0.228763
491 47 estogether 0.228763
491 48 neogether  0.228763
491 49 to ether t 0.228763
491 50 h ether to 0.228763
491 51 nether to  0.228763
491 52 ether to c 0.228763
491 53 nher to co 0.228763
491 54 hem te col 0.228763
491 55 em te coll 0.228763
491 56 n to colle 0.228763
491 57  th collec 0.228763
491 58 to bollect 0.228763
491 59 h bollect  0.228763
491 60 nbollect w 0.228763
491 61 tollect wo 0.228763
491 62 ollect woo 0.228763
491 63 nlect wood 0.228763
491 64 eect wood  0.228763
491 65 est wood a 0.228763
491 66 nt wood an 0.228763
491 67 o wood and 0.228763
491 68 haood and  0.228763
4

493 133  ta long f 0.228763
493 134 to bong fo 0.228763
493 135 h bong for 0.228763
493 136 nbong for  0.228763
493 137 tong for t 0.228763
493 138 eng for th 0.228763
493 139 n' for the 0.228763
493 140 d for the  0.228763
493 141 efor the e 0.228763
493 142 tor the en 0.228763
493 143 or the end 0.228763
493 144 nkthe endl 0.228763
493 145  the endle 0.228763
493 146 toemendles 0.228763
493 147 herendless 0.228763
493 148 emendless  0.228763
493 149 nendless i 0.228763
493 150 tndless im 0.228763
493 151 ndless imm 0.228763
493 152 d ess imme 0.228763
493 153  ess immen 0.228763
493 154 ess immens 0.228763
493 155 ns immensi 0.228763
493 156 iiimmensit 0.228763
493 157 iimmensity 0.228763
493 158 tmmensity  0.228763
493 159 mmensity o 0.228763
493 160  ensity of 0.228763
493 161  nsity of  0.228763
493 162 ndity of t 0.228763
493 163 dity of th 0.228763
493 164 igy of the 0.228763
493 165 my of the  0.228763
493 166 h of the s 0.228763
493 167 oof the se 0.228763
493 168 tf the sea 0

496 8 nd to buil 0.228758
496 9 d to build 0.228758
496 10  to build  0.228758
496 11 to luild a 0.228758
496 12   luild a  0.228758
496 13  luild a s 0.228758
496 14 tutld a sh 0.228758
496 15 utld a shi 0.228758
496 16  ld a ship 0.228758
496 17 td a ship, 0.228758
496 18 e a ship,  0.228758
496 19  a ship, d 0.228758
496 20 tnship, do 0.228758
496 21 nship, don 0.228758
496 22 thip, don' 0.228758
496 23  ip, don't 0.228758
496 24 ep, don't  0.228758
496 25 t, don't d 0.228758
496 26 l don't dr 0.228758
496 27  bon't dru 0.228758
496 28 ton't drum 0.228758
496 29  n't arum  0.228758
496 30  't arum u 0.228758
496 31 dt arum up 0.228758
496 32 t drum up  0.228758
496 33  trum up p 0.228758
496 34 toum up pe 0.228758
496 35  um up peo 0.228758
496 36  m up peop 0.228758
496 37   up peopl 0.228758
496 38  tp people 0.228758
496 39 tp people  0.228758
496 40   people t 0.228758
496 41 lpeople to 0.228758
496 42 teople tog 0.228758
496 43 lople toge 0.228758
496 44 rple toget 0.228758
496

499 45  le togeth 0.228755
499 46 ee togethe 0.228755
499 47 estogether 0.228755
499 48 neogether  0.228755
499 49 to ether t 0.228755
499 50 h ether to 0.228755
499 51  ether to  0.228755
499 52 ether to c 0.228755
499 53 nher to co 0.228755
499 54 hem te col 0.228755
499 55 em te coll 0.228755
499 56 n te colle 0.228755
499 57  th collec 0.228755
499 58 to bollect 0.228755
499 59 h bollect  0.228755
499 60  bollect w 0.228755
499 61 tollect wo 0.228755
499 62 ollect woo 0.228755
499 63  lect wood 0.228755
499 64 eect wood  0.228755
499 65 est wood a 0.228755
499 66 nt wood an 0.228755
499 67 o wood and 0.228755
499 68 haood and  0.228755
499 69 tord and d 0.228755
499 70 ord and do 0.228755
499 71  d and don 0.228755
499 72   and don' 0.228755
499 73  and don't 0.228755
499 74 tnd won't  0.228755
499 75 nd won't a 0.228755
499 76 d won't as 0.228755
499 77  aon't ass 0.228755
499 78 ton't assi 0.228755
499 79  n't assig 0.228755
499 80  't assign 0.228755
499 81 dt assign  0.228755
4

In [4]:
results = sess.run(outputs, feed_dict={X: dataX})
for j, result in enumerate(results):
    index = np.argmax(result, axis=1)
    if j is 0:  # print all for the first result to make a sentence
        print(''.join([char_set[t] for t in index]), end='')
    else:
        print(char_set[index[-1]], end='')

g you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.